In [1]:
!docker compose up -d

[sudo] пароль для alexey: WARN[0000] Found orphan containers ([clickhouse-node4 clickhouse-node1 clickhouse-node3 clickhouse-node2 zookeeper]) for this project. If you removed or renamed this service in your compose file, you can run this command with the --remove-orphans flag to clean it up. 
[+] Running 0/0
 ⠋ Container clickhouse  Creating                                          0.1s 
[+] Running 0/1
 ⠿ Container clickhouse  Starting                                          0.2s 
[+] Running 0/1
 ⠿ Container clickhouse  Starting                                          0.3s 
[+] Running 0/1
 ⠿ Container clickhouse  Starting                                          0.4s 
[+] Running 0/1
 ⠿ Container clickhouse  Starting                                          0.5s 
[+] Running 0/1
 ⠿ Container clickhouse  Starting                                          0.6s 
[+] Running 0/1
 ⠿ Container clickhouse  Starting                                          0.7s 
[+] Running 0/1
 ⠿ Contain

In [15]:
!docker compose down -v

[sudo] пароль для alexey: [+] Running 0/0
 ⠋ Container clickhouse  Stopping                                          0.1s 
[+] Running 0/1
 ⠙ Container clickhouse  Stopping                                          0.2s 
[+] Running 0/1
 ⠹ Container clickhouse  Stopping                                          0.3s 
[+] Running 0/1
 ⠸ Container clickhouse  Stopping                                          0.4s 
[+] Running 0/1
 ⠼ Container clickhouse  Stopping                                          0.5s 
[+] Running 0/1
 ⠴ Container clickhouse  Stopping                                          0.6s 
[+] Running 0/1
 ⠦ Container clickhouse  Stopping                                          0.7s 
[+] Running 0/1
 ⠧ Container clickhouse  Stopping                                          0.8s 
[+] Running 0/1
 ⠇ Container clickhouse  Stopping                                          0.9s 
[+] Running 0/1
 ⠏ Container clickhouse  Stopping                                          1.0s 
[+] 

In [ ]:
import time
import statistics
import uuid
from faker import Faker

from clickhouse_driver import Client

In [2]:
client = Client(host="localhost") 

In [3]:
client.execute("CREATE DATABASE IF NOT EXISTS benchmark")

[]

In [4]:
client.execute("DROP TABLE IF EXISTS benchmark.film_timestamp")

[]

In [5]:
create_sql = """
CREATE TABLE IF NOT EXISTS benchmark.film_timestamp (
      user_id UUID,
      film_id UUID,
      film_timestamp DateTime('Europe/Moscow'),
      event_time DateTime('Europe/Moscow'))
      Engine=MergeTree() PARTITION BY toYYYYMMDD(event_time) order by user_id;
"""
client.execute(create_sql)

[]

In [6]:
client.execute("select count(1) from benchmark.film_timestamp")

[(0,)]

In [9]:
def generate_random_data(num_rows_per_chunk: int, iteration: int):
    chunk = []
    fake = Faker()

    event_time = fake.date_time_between()
    for i in range(iteration*num_rows_per_chunk):
        film_time = fake.date_time_between()
        
        chunk.append((str(uuid.uuid4()), str(uuid.uuid4()), film_time, event_time))

        if len(chunk) == num_rows_per_chunk:
            yield chunk
            chunk = []
            event_time = fake.date_time_between()

    yield chunk

In [10]:
class ProfileCode:
    def __init__(self):
        self.start = time.time()
        self.all_durations = []
        
    def setup_start_time(self):
        self.start = time.time()

    def checkpoint(self):
        end_time = time.time()
        self.all_durations.append(end_time - self.start)
        self.start = end_time

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        print(f"Median - {statistics.median(self.all_durations)}")
        avg = sum(self.all_durations) / len(self.all_durations)
        print(f"Average - {statistics.mean(self.all_durations)}")
        print(f"Summary - {sum(self.all_durations)}")

In [11]:
with ProfileCode() as profiler:
    for chunk in generate_random_data(500, 2000):
        profiler.setup_start_time()
        client.execute(
                    f"INSERT INTO benchmark.film_timestamp  (user_id, film_id, film_timestamp, event_time) VALUES",
                    chunk,
                )
        profiler.checkpoint()

Median - 0.039830684661865234
Average - 0.04410535058398535
Summary - 88.25480651855469


In [12]:
with ProfileCode() as profiler:
    for chunk in generate_random_data(1, 2000):
        profiler.setup_start_time()
        client.execute(
                    f"INSERT INTO benchmark.film_timestamp  (user_id, film_id, film_timestamp, event_time) VALUES",
                    chunk,
                )
        profiler.checkpoint()

Median - 0.008498907089233398
Average - 0.009264890460119671
Summary - 18.539045810699463


In [13]:
num_uuids = 100
uuids = [str(u[0]) for u in client.execute(f"SELECT user_id FROM benchmark.film_timestamp LIMIT {num_uuids}")]

In [14]:
with ProfileCode() as profiler:
    for u in uuids:
        profiler.setup_start_time()
        client.execute(f"SELECT * FROM benchmark.film_timestamp where user_id='{u}'")
        profiler.checkpoint()

Median - 0.23672997951507568
Average - 0.25865265607833865
Summary - 25.865265607833862
